Movie Scripts

In [65]:
from requests import get
from bs4 import BeautifulSoup
import re
import pandas as pd
from time import sleep
from random import randint
from time import time
from IPython.core.display import clear_output
from warnings import warn
from pytube import YouTube
import pytube
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import math
import cv2
import os
import unicodedata
import string

In [66]:
valid_filename_chars = "-_.() %s%s" % (string.ascii_letters, string.digits)
char_limit = 255

def clean_filename(filename, whitelist=valid_filename_chars, replace=' '):
    # replace spaces
    for r in replace:
        filename = filename.replace(r,'_')
    
    # keep only valid ascii chars
    cleaned_filename = unicodedata.normalize('NFKD', filename).encode('ASCII', 'ignore').decode()
    
    # keep only whitelisted chars
    cleaned_filename = ''.join(c for c in cleaned_filename if c in whitelist)
    if len(cleaned_filename)>char_limit:
        print("Warning, filename truncated because it was over {}. Filenames may no longer be unique".format(char_limit))
    return cleaned_filename[:char_limit]    

In [ ]:
# Store data in lists
names = []
runtimes = []
profile_pages = []
release_dates = []
budgets = []
plot_synopsis = []
production_companies = []
spec_eff_comps = []
gross = []
stars = []
directors = []
screen_writers = []
metascores = []
genres = []
mpaa_ratings = []
imdb_ratings = []
movie_awards = []
movie_reviews = []
sub_samples = []
first_weekend_USA = []
full_cast = []

# pages = [str(i) for i in range(1,140000,250)]

pages = [str(i) for i in range(1,140000,250)]


#Prepare monitoring of loop
start_time = time()
requests = 0
#for every page
for page in pages:

    #make get request
    response = get("https://www.imdb.com/search/title?title_type=feature&languages=en&count=250&start=" + page)

    #pause the loop
    sleep(randint(8,15))

    #monitor requests
    requests += 1
    elapsed_time = time() - start_time
    print('Request:{}; Frequency: {} requests/s'.format(requests, requests/elapsed_time))
    clear_output(wait = True)

    #throw a warning for non-200 status codes
    if response.status_code != 200:
        warn('Request: {}; Status code: {}'.format(requests, response.status_code))

    #break the loop if the number of requests is greater than expected
    if requests > 72:
        warn('Number of requests was greater than expected.')
        break

    # parse the content of request
    page_html = BeautifulSoup(response.text, 'html.parser')

    #select all 250 movie containers from a single page
    mv_containers = page_html.find_all('div', class_ = 'lister-item mode-advanced')
#     del mv_containers[:16]
    # Extract data from indiv. movie containers
    for container in mv_containers:
        if container.find('div', class_ = 'ratings-metascore') is not None:
            #Name
            name = container.h3.a.text
            names.append(name)
            
            linkzor = container.h3.a['href']
            
            castz = get("https://www.imdb.com/" + linkzor + "fullcredits")
            # parse the content of request
            cast_html = BeautifulSoup(castz.text, 'html.parser')
            
            try: 
                #cast
                cast_html.find('table', class_ = 'cast_list').findNext('tr').decompose()
                cast_members = []
                cast_odd = cast_html.find('table', class_ = 'cast_list').findAll('tr', class_='odd')
                cast_even = cast_html.find('table', class_ = 'cast_list').findAll('tr', class_='even')
                for cast in cast_odd:
                    cast_members.append(cast.findAll('td')[1].text.strip())
                for cast in cast_even:
                    cast_members.append(cast.findAll('td')[1].text.strip())
                full_cast.append(cast_members)
            except:
                full_cast.append('null')
            
            #Runtime
            runtime = container.find('span', class_ = 'runtime').text
            runtimes.append(runtime)
                        
            #Stars
            movie_stars = []
            stars_x = container.findAll('div')[3].findAll('p')[2].findAll('a')
            del stars_x[0]
            for star in stars_x:
                movie_stars.append(star.text)
            stars.append(movie_stars)

            #Profile
            sleep(randint(1,3))
            profile = container.h3.a['href']
            profile_pages.append(profile)

            details = get("https://www.imdb.com/" + profile)
            # parse the content of request
            details_html = BeautifulSoup(details.text, 'html.parser')
            
            try:
                #opening weekend BO
                bo = details_html.find(text='Opening Weekend USA:').parent.findNext('span').decompose()
                bo = details_html.find(text='Opening Weekend USA:').parent.findNext('span').decompose()
                ow_bo = details_html.find(text='Opening Weekend USA:').parent.parent.text.strip()
                first_weekend_USA.append(ow_bo[20:].replace('$', '').replace(',', '').strip())
            except:
                first_weekend_USA.append("null")

            #save storyline
#             storyline = details_html.find("div", {"id": "titleStoryLine"}).div.span.text
#             storylines.append(storyline)
            
            try:
                #Budget
                bo_budget = details_html.find(text='Budget:').parent.findNext('span').decompose()
                budget = details_html.find(text='Budget:').parent.parent.text.strip()
                budgets.append(budget[7:].replace('$', '').replace(',', ''))
            except:
                budgets.append("null")
                    
            #Release Dates
            html_text = "Release Date:"
            details_html.find(text='Release Date:').parent.findNext('span').decompose()
            date = details_html.find(text='Release Date:').parent.parent.text.strip()
            release_dates.append(re.sub(r'\([^)]*\)', '', date[14:])[:-1])
            
#             movie_name = name
#             award = get("https://www.boxofficemojo.com/oscar/movies/?id={}.htm".format(movie_name.replace(" ", "")))
#             html_awards = BeautifulSoup(award.text, 'html.parser')
#             award_links = html_awards.find("div", {"id": "body"}).find_all('a')
#             awards = {}
#             for award in award_links:
#                 if "(WIN)" in award.text:
#                     link = "https://www.boxofficemojo.com/{}".format(award['href'])
#                     new_url = link
#                     new_response = get(link)
#                     html_award = BeautifulSoup(new_response.text, 'html.parser')
#                     html_award.find('font').extract()
#                     title = html_award.find('font').text
#                     if title == "BEST PICTURE":
#                         awards[title] = movie_name
#                     else:
#                         awards[title] = html_award.find(text=movie_name).findNext('font').text
            
            #Production Companies
            production_comps = []
            sleep(randint(1,3))
            production_link = profile
            production = get("https://www.imdb.com/" + production_link + "companycredits")
            production_html = BeautifulSoup(production.text, 'html.parser')
            comps_x = production_html.find("div", {"id": "company_credits_content"}).ul.findAll('a')
            for comp in comps_x:
                production_comps.append(comp.text)
            production_companies.append(production_comps)
            
            #Special Effects Companies            
            effects = get("https://www.imdb.com/{}companycredits".format(profile))
            effects_html = BeautifulSoup(effects.text, 'html.parser')

            
            if effects_html.find("h4", {"id": "specialEffects"}):
                effects_comps = []
                comps_x = effects_html.find("h4", {"id": "specialEffects"}).findNext('ul').findAll('li')
                for comp in comps_x:
                    effects_comps.append(comp.a.text)
                spec_eff_comps.append(effects_comps)
            else:
                spec_eff_comps.append("null")
            #Director
            director = container.findAll('div')[3].findAll('p')[2].findAll('a')[0].text
            directors.append(director)


            synopsis_link = profile
            sleep(randint(1,3))
            synopsis = get("https://www.imdb.com/" + synopsis_link + "plotsummary")
            synopsis_html = BeautifulSoup(synopsis.text, 'html.parser')

            plot_synopsis_content = synopsis_html.find("ul", {"id": "plot-synopsis-content"}).li.text
            plot_synopsis_content = plot_synopsis_content.strip()
            plot_synopsis.append(plot_synopsis_content)
            
            writers = []
            sleep(randint(1,3))
            credits = get("https://www.imdb.com/{}fullcredits".format(profile))
            credits_html = BeautifulSoup(credits.text, 'html.parser')
            credit_containers = credits_html.find_all('table', class_ = 'simpleTable simpleCreditsTable')
            writers_x = credit_containers[1].tbody.findAll('tr')
            for writer in writers_x:
                if writer.find('td', attrs = {'colspan':'3'}):
                    1==1
                else:
                    answer = writer.find('td', class_ = 'name').text
                    writers.append(answer.strip())
            screen_writers.append(writers)
            
            
            #Metascore
            metascore = container.find('span', class_ = 'metascore')
            metascore = int(metascore.text)
            metascores.append(metascore)

            
            #Genre
            genre = container.p.find('span', class_ = 'genre').text.strip()
            genres.append(genre)
            
            if container.p.find('span', class_ = 'certificate') is not None:
                #MPAA rating
                mpaa = container.p.find('span', class_ = 'certificate').text
                mpaa_ratings.append(mpaa)
            else:
                na = "N/A"
                mpaa_ratings.append(na)
                
            #IMDb rating
            imdb = float(container.strong.text)
            imdb_ratings.append(imdb)
            
            try:
                #Gross
                bo_gross = container.findAll('div')[3].findAll('p')[3].findAll('span')[4]
                bo_gross = bo_gross['data-value']
                bo_gross = re.sub("[^\d\.]", "", bo_gross)
                gross.append(int(bo_gross))
            except:
                gross.append("null")
                
            comments = []
            reviews_link = profile
            all_reviews = get("https://www.imdb.com/" + reviews_link + "reviews")
            html_reviews = BeautifulSoup(all_reviews.text, 'html.parser')
            num_reviews = int(html_reviews.find("div", {"id": "main"}).section.find('div', class_ = 'lister').span.text[:-7].strip().replace(',', ''))
            math.ceil(num_reviews/25) -1

    
            driver = webdriver.Chrome("C:\\Users\Brian\Downloads\chromedriver\chromedriver.exe")
            driver.get("https://www.imdb.com/" + reviews_link + "reviews")
            sleep(randint(3,3))
            try:
                button = driver.find_element_by_id('load-more-trigger')
                for i in range(math.ceil(num_reviews/25) - 1):
                    button.click()
                    sleep(randint(1,1))
            except:
                print('{} only has 1 page of movie reviews'.format(name))
            src = driver.page_source
            parser = BeautifulSoup(src, 'lxml')
            list_of_attributes = {"class":"lister-item"}
            review_containers = parser.findAll("div", attrs=list_of_attributes)
            for container in review_containers:
                comments.append(container.div.div.find("div", class_ = "content").div.text)
            movie_reviews.append(comments)

            driver.close()
            
            #Trailer
            movie_name = name
            response = get("https://www.google.com/search?q={}+Trailer".format(movie_name.replace(" ", "+")))
            html_google = BeautifulSoup(response.text, 'html.parser')

            try:
                txt = html_google.find('div', class_='g').text
                x = re.findall("(http|ftp|https)://([\w_-]+(?:(?:\.[\w_-]+)+))([\w.,@?^=%&:/~+#-]*[\w@?^=%&/~+#-])?",txt)
                link = "{}://{}{}".format(x[0][0], x[0][1], x[0][2])
                yt = pytube.YouTube(link)
                stream = yt.streams.first()
                os.mkdir("C:\\Users\Brian\OneDrive - Southern Connecticut State University\Movie_Data\Movie_Trailers\{}".format(clean_filename(movie_name)))
                stream.download("C:\\Users\Brian\OneDrive - Southern Connecticut State University\Movie_Data\Movie_Trailers\{}".format(clean_filename(movie_name)), "{}_trailer".format(clean_filename(movie_name)))
            except:
                print('No Trailer for {}'.format(name))
#             #Sub Sample Extraction
#             def FrameCapture(path): 

#                 # Path to video file 
#                 vidObj = cv2.VideoCapture(path) 

#                 # Used as counter variable 
#                 count = 0

#                 # checks whether frames were extracted 
#                 success = 1

#                 while success: 
#                     # vidObj object calls read 
#                     # function extract frames 
#                     success, image = vidObj.read() 

#                     # Saves the frames with frame-count
#                     if count % 2 == 0:
#                         cv2.imwrite("C:\\Users\\Brian\\Desktop\\IMDb_Scrape\\Movie_Trailers\\{}\\frame%d.jpeg".format(clean_filename(movie_name)) % count, image) 

#                     count += 1
#             try:
#                 if __name__ == '__main__': 

#                     # Calling the function 
#                     FrameCapture("C:\\Users\\Brian\\Desktop\\IMDb_Scrape\\Movie_Trailers\\{}\\{}_trailer.mp4".format(clean_filename(movie_name), clean_filename(movie_name))) 
#             except:
#                 sub_samples.append(clean_filename(movie_name))

No Trailer for Dumbo
The Dirt only has 1 page of movie reviews
No Trailer for The Dirt
No Trailer for Escape Room
Avengers: Infinity War only has 1 page of movie reviews
No Trailer for Avengers: Infinity War
No Trailer for Spider-Man: Into the Spider-Verse
No Trailer for Pet Sematary
No Trailer for The Mule
No Trailer for Bumblebee
No Trailer for Unplanned
No Trailer for Triple Frontier


In [82]:
# Store data in lists
names = []
full_cast = []

# pages = [str(i) for i in range(1,141000,250)]
pages = ['1']

#Prepare monitoring of loop
start_time = time()
requests = 0
#for every page
for page in pages:

    #make get request
    response = get("https://www.imdb.com/search/title?title_type=feature&languages=en&count=50&start=" + page)

    #pause the loop
    sleep(randint(8,15))

    #monitor requests
    requests += 1
    elapsed_time = time() - start_time
    print('Request:{}; Frequency: {} requests/s'.format(requests, requests/elapsed_time))
    clear_output(wait = True)

    #throw a warning for non-200 status codes
    if response.status_code != 200:
        warn('Request: {}; Status code: {}'.format(requests, response.status_code))

    #break the loop if the number of requests is greater than expected
    if requests > 72:
        warn('Number of requests was greater than expected.')
        break

    # parse the content of request
    page_html = BeautifulSoup(response.text, 'html.parser')

    #select all 250 movie containers from a single page
    mv_containers = page_html.find_all('div', class_ = 'lister-item mode-advanced')
    del mv_containers[:25]
    # Extract data from indiv. movie containers
    for container in mv_containers:
        if container.find('div', class_ = 'ratings-metascore') is not None:
            #Name
            name = container.h3.a.text
            names.append(name)
            
            linkzor = container.h3.a['href']
            
            castz = get("https://www.imdb.com/" + linkzor[:-15] + "fullcredits")
            # parse the content of request
            cast_html = BeautifulSoup(castz.text, 'html.parser')
            
            #cast
            cast_html.find('table', class_ = 'cast_list').findNext('tr').decompose()
            cast_members = []
            cast_odd = cast_html.find('table', class_ = 'cast_list').findAll('tr', class_='odd')
            cast_even = cast_html.find('table', class_ = 'cast_list').findAll('tr', class_='even')
            for cast in cast_odd:
                cast_members.append(cast.findAll('td')[1].text.strip())
            for cast in cast_even:
                cast_members.append(cast.findAll('td')[1].text.strip())
            full_cast.append(cast_members)


Request:1; Frequency: 0.08826853323335623 requests/s


In [83]:
print("Names Length: {}".format(len(names)))
print("Cast Length: {}".format(len(full_cast)))
# print("Opening Weekend: {}".format(len(first_weekend_USA)))
# print("Length: {}".format(len(runtimes)))
# print("Profile: {}".format(len(profile_pages)))
# print("Spe. Eff.: {}".format(len(spec_eff_comps)))
# print("Date: {}".format(len(release_dates)))
# print("Budget: {}".format(len(budgets)))
# print("Synopsis: {}".format(len(plot_synopsis)))
# print("Pro. Comp: {}".format(len(production_companies)))
# print("Gross Length: {}".format(len(gross)))
# print("Stars: {}".format(len(stars)))
# print("Director Length: {}".format(len(directors)))
# print("Screen Writers Length: {}".format(len(screen_writers)))
# print("Metascore: {}".format(len(metascores)))
# print("Genre: {}".format(len(genres)))
# print("MPAA: {}".format(len(mpaa_ratings)))
# print("IMDB: {}".format(len(imdb_ratings)))
# print("Comments: {}".format(len(movie_reviews)))

Names Length: 17
Cast Length: 17


In [84]:
movie_ratings = pd.DataFrame({'movie': names,
                              'cast': full_cast
#                               'first_weekend': first_weekend_USA
#                               'length': runtimes,
# #                               'profile': profile_pages,
#                               'date': release_dates,
#                               'budget': budgets,
#                               'synopsis': plot_synopsis,
#                               'pro. comp': production_companies,
#                               'spec. eff': spec_eff_comps,
#                                 'gross': gross,
#                               'stars': stars,
#                         'director': directors,
#                         'writer': screen_writers,
#                              'metascore': metascores,
#                               'genre': genres,
#                               'mpaa': mpaa_ratings,
#                               'imdb': imdb_ratings,
#                               'reviews': movie_reviews
                             })

print(movie_ratings.info())
movie_ratings.head(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17 entries, 0 to 16
Data columns (total 2 columns):
movie    17 non-null object
cast     17 non-null object
dtypes: object(2)
memory usage: 352.0+ bytes
None


,movie,cast
0,Mary Poppins Returns,"[Emily Blunt, Ben Whishaw, Pixie Davies, Joel ..."
1,The Favourite,"[Olivia Colman, Emma Delves, Emma Stone, Jenni..."
2,Get Out,"[Daniel Kaluuya, Catherine Keener, Caleb Landr..."
3,Deadpool 2,"[Ryan Reynolds, Morena Baccarin, Zazie Beetz, ..."
4,How to Train Your Dragon: The Hidden World,"[Jay Baruchel, F. Murray Abraham, Gerard Butle..."
5,Mortal Engines,"[Hera Hilmar, Hugo Weaving, Ronan Raftery, Pat..."
6,Instant Family,"[Mark Wahlberg, Isabela Moner, Julianna Gamiz,..."
7,Vice,"[Christian Bale, Steve Carell, Alison Pill, Ju..."
8,Wonder Park,"[Sofia Mali, Ken Hudson Campbell, Mila Kunis, ..."
9,Stan & Ollie,"[Steve Coogan, Shirley Henderson, Rufus Jones,..."


In [18]:
url = "https://www.imdb.com/title/tt1727824/companycredits"
response = get(url)

effects_html = BeautifulSoup(response.text, 'html.parser')

effects_comps = []
comps_x = effects_html.find("h4", {"id": "specialEffects"}).findNext('ul').findAll('li')
for comp in comps_x:
    effects_comps.append(comp.a.text)
effects_comps

['Double Negative (DNEG)',
 'Halo VFX',
 'Argon Effects',
 'Clear Angle Studios',
 'Centroid Motion Capture',
 'The Visual Effects Company',
 'Fangs FX']

In [142]:
movie_name = "Alita: Battle Angel"
movie_year = "2019"
url_1 = "https://www.boxofficemojo.com/oscar/movies/?id={}.htm".format(movie_name.replace(" ", ""))
response_1 = get(url_1)
html_awards = BeautifulSoup(response_1.text, 'html.parser')
if html_awards.find("div", {"id": "body"}).h1.text not in [movie_name, "{} ({})".format(movie_name, movie_year)]:
    url_1 = "https://www.boxofficemojo.com/oscar/movies/?id={}{}.htm".format(movie_name.replace(" ", ""),movie_year)
    response_1 = get(url_1)
    html_awards = BeautifulSoup(response_1.text, 'html.parser')
    blah = []
    award_links = html_awards.find("div", {"id": "body"}).find_all('a')
    awards = {}
    for award in award_links:
        if "(WIN)" in award.text:
            link = "https://www.boxofficemojo.com/{}".format(award['href'])
            new_url = link
            new_response = get(link)
            html_award = BeautifulSoup(new_response.text, 'html.parser')
            html_award.find('font').extract()
            title = html_award.find('font').text
            if title == "BEST PICTURE":
                awards[title] = movie_name
            else:
                movie_name
                awards[title] = html_award.find(text="{} ({})".format(movie_name, movie_year)).findNext('font').text
else:
    award_links = html_awards.find("div", {"id": "body"}).find_all('a')
    awards = {}
    for award in award_links:
        if "(WIN)" in award.text:
            link = "https://www.boxofficemojo.com/{}".format(award['href'])
            new_url = link
            new_response = get(link)
            html_award = BeautifulSoup(new_response.text, 'html.parser')
            html_award.find('font').extract()
            title = html_award.find('font').text
            if title == "BEST PICTURE":
                awards[title] = movie_name
            else:
                awards[title] = html_award.find(text=movie_name).findNext('font').text
print(awards)

AttributeError: 'NoneType' object has no attribute 'findNext'

In [51]:
comments = []


url = "https://www.imdb.com/title/tt7343762/reviews"
response = get(url)
html_reviews = BeautifulSoup(response.text, 'html.parser')
# print(html_reviews.find("div", {"id": "main"}))
# print(html_reviews.find("div", {"id": "main"}).section)
# print(html_reviews.find("div", {"id": "main"}).section.find('div', class_ = 'lister').span)
print(html_reviews.find("div", {"id": "main"}).section.find('div', class_ = 'lister').span.text[:-7].strip().replace(',', ''))
# num_reviews = int(html_reviews.find("div", {"id": "main"}).section.find('div', class_ = 'lister').span.text[:-8].replace(',', ''))
# math.ceil(num_reviews/25) -1


# driver = webdriver.Chrome("C:\\Users\Brian\Downloads\chromedriver\chromedriver.exe")
# driver.get("https://www.imdb.com/title/tt7343762/reviews")
# try:
#     button = driver.find_element_by_id('load-more-trigger')
#     for i in range(math.ceil(num_reviews/25) - 1):
#         button.click()
#         sleep(randint(1,1))
# except:
#     print("Only 1 page of reviews")
# src = driver.page_source
# parser = BeautifulSoup(src, 'lxml')
# list_of_attributes = {"class":"lister-item"}
# review_containers = parser.findAll("div", attrs=list_of_attributes)
# for container in review_containers:
#     comments.append(container.div.div.find("div", class_ = "content").div.text)
# driver.close()

1


In [25]:
comments

["I hate how people in horror movies do dumb stuff. Like go upstairs instead of trying to use the back door and making it outside. The characters do the same thing here specifically the kids. It's frustrating on how bad the writing was. Like why would you do that? It has so much potential. Cinematography and production really stand out for me. The pale colors and the set pieces I really enjoy. The story is as simple as it gets. CPS case worker goes to discover 2 children dead in a canal. She has to take her children with her, even though the boy is old enough to babysit. Then the boy goes on an adventure and wants to be nosy. Then he encounters The vengeful spirit La Llarona and gets his sister and mother cursed. Then the mom looks for help to stop her from taking her children. I swear I've seen this before. The Jump scares still kind of get you, even though you see them coming. It's the same old, same old. Im very worried for the conjuring 3 now. I hope they get new writers.",
 "I'm a

In [69]:
pages = [str(i) for i in range(15001,140000,250)]
pages

['15001',
 '15251',
 '15501',
 '15751',
 '16001',
 '16251',
 '16501',
 '16751',
 '17001',
 '17251',
 '17501',
 '17751',
 '18001',
 '18251',
 '18501',
 '18751',
 '19001',
 '19251',
 '19501',
 '19751',
 '20001',
 '20251',
 '20501',
 '20751',
 '21001',
 '21251',
 '21501',
 '21751',
 '22001',
 '22251',
 '22501',
 '22751',
 '23001',
 '23251',
 '23501',
 '23751',
 '24001',
 '24251',
 '24501',
 '24751',
 '25001',
 '25251',
 '25501',
 '25751',
 '26001',
 '26251',
 '26501',
 '26751',
 '27001',
 '27251',
 '27501',
 '27751',
 '28001',
 '28251',
 '28501',
 '28751',
 '29001',
 '29251',
 '29501',
 '29751',
 '30001',
 '30251',
 '30501',
 '30751',
 '31001',
 '31251',
 '31501',
 '31751',
 '32001',
 '32251',
 '32501',
 '32751',
 '33001',
 '33251',
 '33501',
 '33751',
 '34001',
 '34251',
 '34501',
 '34751',
 '35001',
 '35251',
 '35501',
 '35751',
 '36001',
 '36251',
 '36501',
 '36751',
 '37001',
 '37251',
 '37501',
 '37751',
 '38001',
 '38251',
 '38501',
 '38751',
 '39001',
 '39251',
 '39501',
 '39751',


In [52]:
list = [1,2,3,4,5,6,7,8,9,10]
del list[:5]

In [53]:
list

[6, 7, 8, 9, 10]

In [64]:
len(names)

715